In [8]:
import pandas as pd

In [10]:
df = pd.read_excel('./module_2_files/data.xlsx')

In [24]:
df.head()

,id,date,user_id,duration,medium,source,cost,order_id,amount_paid
0,40443,2016-10-05 23:18:00,1010,0.000926,seo,google,0.00,6243,20.2
1,35044,2016-10-09 21:40:00,1036,0.006493,sem,yandex,0.07,6145,15.6
2,40177,2016-10-05 03:23:00,1041,0.003380,email,promo,0.00,6128,13.2
3,39401,2016-10-05 23:19:00,1041,0.000463,sem,yandex,0.03,6697,9.8
4,41545,2016-10-01 04:57:00,1042,0.006493,sem,google,0.06,4510,14.8


# ДОМАШНЕЕ ЗАДАНИЕ
Файл data_no_header.txt

**ROI = отношение прибыли (amount_paid - cost) к расходам cost**

Вам необходимо посчитать ROI в разрезе типов источников трафика. Т. е. получить значения ROI не по источника трафика как в прошлом блоке, а по типам. Типы определяются следующим образом:

    - если source равен 'google' или 'yandex', то проверяем medium:
        - для medium 'seo' или 'sem' тип источника равен 'search engines seo'
        - для medium 'brand' - тип источника равен 'search engines brand'
        - для остальных случаев тип источника равен 'search engines undefined'
    - если условие не выполнено, то тип источника равен 'other'

Пожалуйста, учитывайте, что все условия на партнерские комиссии сохраняются на домашнее задание.

In [1]:
cpa_commission = {
    'burgerclub': 0.3,
    'food-delivery': 0.25
}

cpc_commission = {
    'city-magazine': 7,
    'foody': 9
}

fixed_commission = 4

In [25]:
def source_type_clf2(line):
    """
    Пример строки на входе функции:
    ['40443', '05.10.2016 23:18', '1010', '0,000925926', 'seo', 'google', '0', '6243', '20,20']
    Функция классифицирует источник трафика в зависимости от значения source и medium.
    Возвращает название типа источника по правилам:
        - если source равен 'google' или 'yandex', то проверяем medium:
            - для medium 'seo' или 'sem' возвращаем 'search engines seo'
            - для medium 'brand' - возвращаем 'search engines brand'
            - для остальных случаев возвращаем 'search engines undefined'
        - если условие не выполнено, то возвращаем 'other'
    """
    source = line[5]
    medium = line[4]

    if source == 'google' or source == 'yandex':
        if medium == 'seo' or medium == 'sem':
            return 'search engines seo'
        elif medium == 'brand':
            return 'search engines brand'
        else:
            # если вдруг встретится другой вариант, то ставим "undefined"
            return 'search engines undefined'
    else:
        return 'other'

In [28]:
def costs_classification( amount_paid, source ):
    """
    Функция по названию канала source возвращает размер комиссии.
    Размеры комиссий берутся из словарей cpa_commission, cpc_commission и значения fixed_commission
    """
    # если источник source входит в словарь cpa_commission, то возвращаем долю от выручки
    if source in cpa_commission:
        return amount_paid * cpa_commission[ source ]
    # в случае партнеров CPC стоимость расходов постоянная
    if source in cpc_commission:
        return cpc_commission[ source ]
    # если ни один случай не подходит, то возвращаем фиксированную стоимость fixed_commission
    return fixed_commission

In [40]:
def expense_and_income( line ):
    """
    Функция для строки line возвращает итоговый расход и доход
    """
    source = line[5]
    source_type = source_type_clf2(line)
    amount_paid = float( line[-1].replace( ',', '.' ) )
    cost = float( line[6].replace( ',', '.' ) )
    partner_comission = costs_classification( amount_paid, source )
    return source_type, cost + partner_comission, amount_paid

In [43]:
roi_stats = {}

with open('./module_2_files/data_no_header.txt', 'r') as f:
    for line in f:
        line = line.strip().split('\t')
#         print(expense_and_income(line))
        source_type, cost, income = expense_and_income(line)
        if source_type in roi_stats:
            roi_stats[source_type]['cost'] += cost
            roi_stats[source_type]['income'] += income
        else:
            roi_stats[source_type] = {}
            roi_stats[source_type]['cost'] = cost
            roi_stats[source_type]['income'] = income

In [44]:
roi_stats

{'search engines seo': {'cost': 686.2699999999992, 'income': 2902.4},
 'other': {'cost': 541.1059999999999, 'income': 1992.7999999999997},
 'search engines brand': {'cost': 52.0, 'income': 234.60000000000005}}

In [47]:
for source, data in roi_stats.items():
    data['roi'] = (data['income'] - data['cost']) / data['cost']
    print('Source {} ROI {:.2f}'.format(source, data['roi']))

Source search engines seo ROI 3.23
Source other ROI 2.68
Source search engines brand ROI 3.51
